In [ ]:
import PIL
from PIL import Image

!unzip facedataset.zip

import os
training_list = os.listdir("FER_2013 dataset copy/train")
print(len(training_list))

#training set

class_labels = ['angry', 'happy', 'neutral', 'sad']

fw = open('training_list.txt','w')

for dirpath, dirname, filenames in os.walk('FER_2013 dataset/train'):
  if 'FER_2013 dataset/train' is not dirpath:
    print(dirpath)
    dir_name = dirpath.split('/')[-1]
    print(dir_name)

    if dir_name in class_labels:
      for f in filenames [:3000]:
        print(dirpath, f)

        fw.write(dirpath + '/' + f + '\n')
        
fw.close()

#testing set

import os
testing_list = os.listdir("FER_2013 dataset copy/test")
print(len(testing_list))
print(testing_list)

class_labels = ['angry', 'happy', 'neutral', 'sad']

fw = open('testing_list.txt','w')

for dirpath, dirname, filenames in os.walk('FER_2013 dataset/test'):
  if 'FER_2013 dataset/test' is not dirpath:
    print(dirpath)
    dir_name = dirpath.split('/')[-1]
    print(dir_name) 

    if dir_name in class_labels:
      for f in filenames [:500]:
        print(dirpath, f)

        fw.write(dirpath + '/' + f + '\n')




from keras.preprocessing.image import ImageDataGenerator
my_generator = ImageDataGenerator(rescale=1./255)
train_generator = my_generator.flow_from_directory('FER_2013 dataset copy/train/', target_size=(48,48), batch_size=16, class_mode='categorical')
validate_generator = my_generator.flow_from_directory('FER_2013 dataset copy/test', target_size=(48,48), batch_size=16, class_mode='categorical')



import matplotlib.pyplot as plt
for my_batch in train_generator:
    images = my_batch[0]
    labels = my_batch[1]
    for i in range(len(labels)):
      plt.imshow(images[i])
      plt.colorbar()
      plt.show()
      print(images[i].shape)
      print(labels[i])
      break
    break

#deep learning convolutional neural network for image classification/machine learning

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
# the first argument represents number of input features. bigger number = more broad, smaller number = more detailed 
# padding arg makes the dimensions in the first convolved layer the same as original dimensions (48x48)
model = Sequential()
model.add( Conv2D( 64, ( 3, 3 ), activation = 'relu', input_shape = (48, 48, 3) ,  padding = 'same' ) )
model.add( BatchNormalization() ) 
model.add( Conv2D( 64, ( 3, 3 ), activation = 'relu', padding = 'same' ) )
model.add( BatchNormalization())
model.add( MaxPool2D(2,2) )
model.add( Conv2D( 32, ( 3, 3 ), activation = 'relu', padding = 'same' ) )
model.add(Dropout(rate=0.4) )
model.add( Conv2D( 32, ( 3, 3 ), activation = 'relu', padding = 'same' ) )
model.add(BatchNormalization() )
model.add( MaxPool2D(2,2) )
model.add( Conv2D( 16, ( 3, 3 ), activation = 'relu', padding = 'same' ) )
model.add(BatchNormalization() )
model.add( Conv2D( 16, ( 3, 3 ), activation = 'relu', padding = 'same' ) )
model.add(Dropout(rate = 0.4) )
model.add( MaxPool2D(2,2) )
model.add( Conv2D( 8, ( 3, 3 ), activation = 'relu', padding = 'same' ) )
model.add(BatchNormalization() )
model.add( Conv2D( 8, ( 3, 3 ), activation = 'relu', padding = 'same' ) )
model.add(BatchNormalization() )
model.add( Flatten() )
# dense layer needs it to be converted into single dimensional
model.add( Dense( 16, activation = 'relu' ) )
model.add( Dense( 4, activation = 'softmax' ) )
model.summary()

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, BatchNormalization


base_model = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(48, 48, 3),
)

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add( Dense( 32, activation = 'relu' ) )
model.add(BatchNormalization())
model.add( Dense( 10, activation = 'relu' ) )
model.add( Dense( 4, activation = 'softmax' ) )
model.summary()



model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [ 'accuracy' ] )

from tensorflow.keras.callbacks import ModelCheckpoint
history = model.fit( train_generator, validation_data = validate_generator, epochs = 50, batch_size = 16, callbacks = [ModelCheckpoint(filepath = 'model_weights.hdf5', monitor = 'val_loss', save_best_only = True, save_weights_only = True, verbose = 1 )] )

import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.ylim(0, 1.2)
plt.legend()
plt.show()
plt.show()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu', input_shape = (48,48,3)))

for i in range(16):
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 4, kernel_size = 3, activation = 'relu', padding='same'))
    model.add(Dropout(rate=0.4))
    model.add(Conv2D(filters = 4, kernel_size = 3, activation = 'relu', padding='same'))

model.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = 2, strides = 2))
model.add(Flatten())
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 4, activation = 'softmax'))

print(model.summary())

model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [ 'accuracy' ] )

from tensorflow.keras.callbacks import ModelCheckpoint
history = model.fit( train_generator, validation_data = validate_generator, epochs = 20, batch_size = 16, callbacks = [ModelCheckpoint(filepath = 'newmodel_weights.hdf5', monitor = 'val_loss', save_best_only = True, save_weights_only = True, verbose = 1 )] )

def plot_learning_curves(history):
    # print(history.params)
    plt.clf()
    if 'accuracy' in history.history:
        plt.plot(history.history['accuracy'], 'g', label = 'Training Accuracy')
        plt.plot(history.history['val_accuracy'], 'b', label = 'Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
    
    plt.ylim(0, 1.2)
    plt.legend()
    plt.show()

plot_learning_curves(history)

<>:30: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:53: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:30: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:53: SyntaxWarning: "is not" with a literal. Did you mean "!="?


FileNotFoundError: ignored